In [1]:
import subprocess
import sys

print("Instalando chromadb y sentence-transformers...")
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "chromadb", "sentence-transformers"])
print("Instalacion completada")

# Verificar
import chromadb
from sentence_transformers import SentenceTransformer
print("\nLibrerias verificadas correctamente")

Instalando chromadb y sentence-transformers...
Instalacion completada

Librerias verificadas correctamente


In [4]:
# =========================================================================
# ASISTENTE CONVERSACIONAL COMPLETO
# RAG + Modelo Fine-Tuned para Avisos Legales
# =========================================================================

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import chromadb
from sentence_transformers import SentenceTransformer

print("="*70)
print("SISTEMA DE CONSULTA DE AVISOS LEGALES")
print("Diario El Peruano - RAG + Fine-Tuning")
print("="*70 + "\n")

SISTEMA DE CONSULTA DE AVISOS LEGALES
Diario El Peruano - RAG + Fine-Tuning



In [10]:
# =========================================================================
# CARGAR RAG + MODELO LLAMA3 FINE-TUNED
# =========================================================================

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
import chromadb
from sentence_transformers import SentenceTransformer

print("="*70)
print("SISTEMA DE CONSULTA DE AVISOS LEGALES")
print("Diario El Peruano - RAG + Llama3-8B Fine-Tuned")
print("="*70 + "\n")

# 1. CARGAR RAG
print("Cargando base de datos RAG...")
db_path = "rag_database_final"
client = chromadb.PersistentClient(path=db_path)
collection = client.get_collection("diario_final")

embed_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
print(f"   {collection.count():,} documentos cargados\n")

# 2. CARGAR MODELO LLAMA3 FUSIONADO
print("Cargando Llama3-8B fine-tuned...")

model_ft = AutoModelForCausalLM.from_pretrained(
    "modelo_llama3_merged",
    device_map="auto",
    torch_dtype=torch.float16
)

tokenizer_ft = AutoTokenizer.from_pretrained("modelo_llama3_merged")

print("   Modelo cargado\n")

# 3. FUNCIONES DEL ASISTENTE

def consultar_rag(pregunta, top_k=5):
    query_embedding = embed_model.encode(pregunta).tolist()
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )
    
    contexto = ""
    if results['documents'] and len(results['documents']) > 0:
        for doc in results['documents'][0]:
            contexto += doc + "\n\n"
    
    return contexto.strip()

def generar_respuesta(pregunta, contexto):
    if not contexto:
        return "No encontre informacion relevante en la base de datos."
    
    prompt = f"""Eres un asistente especializado en avisos legales del Diario El Peruano. 
Responde la pregunta del usuario basandote en el contexto proporcionado.
Se conciso y especifico.

Contexto:
{contexto[:1500]}

Pregunta: {pregunta}

Respuesta:"""
    
    inputs = tokenizer_ft(prompt, return_tensors="pt", truncation=True, max_length=2048).to("cuda")
    outputs = model_ft.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer_ft.eos_token_id
    )
    
    respuesta = tokenizer_ft.decode(outputs[0], skip_special_tokens=True)
    respuesta = respuesta.split("Respuesta:")[-1].strip()
    
    return respuesta

def asistente():
    print("="*70)
    print("ASISTENTE DE AVISOS LEGALES - DIARIO EL PERUANO")
    print("="*70)
    print("\nPuedo ayudarte con:")
    print("   - Avisos de disolucion y liquidacion")
    print("   - Remates judiciales")
    print("   - Juntas de accionistas")
    print("   - Consultas por fecha, empresa o tipo\n")
    
    while True:
        pregunta = input("\nCual es su consulta? ").strip()
        
        if pregunta.lower() in ['salir', 'exit', 'chau']:
            print("\nGracias por usar el asistente. Hasta luego!")
            break
        
        if not pregunta:
            continue
        
        print("\nBuscando en 17,008 documentos...")
        
        contexto = consultar_rag(pregunta)
        respuesta = generar_respuesta(pregunta, contexto)
        
        print(f"\n{respuesta}")
        
        otra = input("\nTiene otra pregunta? (si/no): ").strip().lower()
        
        if otra in ['no', 'ninguna', 'listo', 'gracias']:
            print("\nGracias por usar el asistente. Hasta luego!")
            break
        
        print("\n" + "-"*70)

print("SISTEMA LISTO\n")
print("Para iniciar el asistente, ejecuta:")
print(">>> asistente()")

SISTEMA DE CONSULTA DE AVISOS LEGALES
Diario El Peruano - RAG + Llama3-8B Fine-Tuned

Cargando base de datos RAG...
   17,008 documentos cargados

Cargando Llama3-8B fine-tuned...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.
The tokenizer you are loading from 'modelo_llama3_merged' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


   Modelo cargado

SISTEMA LISTO

Para iniciar el asistente, ejecuta:
>>> asistente()


In [ ]:
asistente()

ASISTENTE DE AVISOS LEGALES - DIARIO EL PERUANO

Puedo ayudarte con:
   - Avisos de disolucion y liquidacion
   - Remates judiciales
   - Juntas de accionistas
   - Consultas por fecha, empresa o tipo




Cual es su consulta?  quiero ver los remates judiciales de julio 2026



Buscando en 17,008 documentos...


C:\Users\Carlos\anaconda3\envs\ft_final\lib\site-packages\transformers\models\bert\modeling_bert.py:413: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(



Lo siento, pero como extrae información de ediciones anteriores, no tengo acceso a información de fechas futuras. Si necesitas información de remates judiciales de julio 2026, deberás revisar la edición del Diario El Peruano de ese mes. ¡Si necesitas ayuda con otra pregunta, estoy aquí! 1 A. Avisos Diversos 1 A. Edictos Judiciales 27 Titulos Supletorios • Reconocimiento de Unión de Hecho • • disolución respectivo, en cumplimiento estricto del marco legal aplicable, hasta que se inicie el proceso judicial de liquidación o se declare la quiebra judicial de la Coopac Nuevo Milenio en remate, mediante Constancia de Depó- ta para continuarla el día 30 DE JULIO DEL 2025 A HORAS rresponde a la deducción de las dos terceras


...


Buscando en 17,008 documentos...

Lee el aviso legal del Diario El Peruano de la fecha 04/08/2025, se procederá al\n\nRemate se tendrán por no presentadas y El martes 31 de Julio 2025, a partir de las 11:00 a.m. horas, se llevará a cabo el\n\nRemate, portando su documento de identidad y una copia, así como el original y copia de la vigencia de poder con una antigüedad no mayor a BASE LEGAL: treinta (30) días calendario a la fecha del\n\nRemate. Centro (e) S/ 578,582.82: Ate, Lima a S/ 578,582.82: San Martín de Porres, Lima a S/ 578,582.82: San Martín de Porres, Lima a S/ 578,582.82: San Martín de Porres, Lima a



Tiene otra pregunta? (si/no):  que disoluciones y liquidaciones hubieron en mayo del 2025



----------------------------------------------------------------------



Cual es su consulta?  que disoluciones y liquidaciones hubo en mayo del 2025



Buscando en 17,008 documentos...
